# Tutorial 21: Advanced Research Topics

This tutorial explores cutting-edge research topics in deep learning, including neural ODEs, implicit neural representations, self-supervised learning, and other emerging techniques.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Optional, Dict
import time
from torchdiffeq import odeint_adjoint as odeint  # For neural ODEs

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Neural Ordinary Differential Equations (Neural ODEs)

Neural ODEs represent neural networks as continuous dynamical systems, allowing for memory-efficient training and continuous-time modeling.

In [ ]:
class ODEFunc(nn.Module):
    """ODE function for Neural ODE"""
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, dim)
        )
        
    def forward(self, t, x):
        return self.net(x)

class NeuralODE(nn.Module):
    """Neural ODE block"""
    def __init__(self, func, t0=0.0, t1=1.0):
        super().__init__()
        self.func = func
        self.t = torch.tensor([t0, t1]).float()
        
    def forward(self, x):
        # Solve ODE
        self.func.nfe = 0  # Number of function evaluations
        out = odeint(self.func, x, self.t, method='dopri5')
        return out[1]  # Return final state

In [ ]:
# Create spiral dataset for classification
def create_spiral_data(n_samples=1000, noise=0.1):
    """Create spiral dataset for classification"""
    t = torch.linspace(0, 4 * np.pi, n_samples // 2)
    
    # Class 0: clockwise spiral
    x0 = t * torch.cos(t) + noise * torch.randn(n_samples // 2)
    y0 = t * torch.sin(t) + noise * torch.randn(n_samples // 2)
    
    # Class 1: counter-clockwise spiral
    x1 = -t * torch.cos(t) + noise * torch.randn(n_samples // 2)
    y1 = t * torch.sin(t) + noise * torch.randn(n_samples // 2)
    
    X = torch.stack([
        torch.cat([x0, x1]),
        torch.cat([y0, y1])
    ], dim=1)
    
    y = torch.cat([
        torch.zeros(n_samples // 2),
        torch.ones(n_samples // 2)
    ]).long()
    
    return X, y

# Generate data
X_spiral, y_spiral = create_spiral_data(n_samples=500)

# Visualize data
plt.figure(figsize=(8, 6))
colors = ['blue', 'red']
for i in range(2):
    mask = y_spiral == i
    plt.scatter(X_spiral[mask, 0], X_spiral[mask, 1], 
                c=colors[i], alpha=0.6, label=f'Class {i}')
plt.title('Spiral Dataset')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Build and train Neural ODE classifier
class ODEClassifier(nn.Module):
    """Classifier using Neural ODE"""
    def __init__(self, input_dim=2, hidden_dim=64, num_classes=2):
        super().__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.ode = NeuralODE(ODEFunc(hidden_dim))
        self.decoder = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.ode(x)
        x = self.decoder(x)
        return x

# Train Neural ODE
print("Training Neural ODE classifier...")
ode_model = ODEClassifier().to(device)
optimizer = optim.Adam(ode_model.parameters(), lr=0.01)

X_train = X_spiral.to(device)
y_train = y_spiral.to(device)

losses = []
for epoch in range(100):
    optimizer.zero_grad()
    logits = ode_model(X_train)
    loss = F.cross_entropy(logits, y_train)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    
    if epoch % 20 == 0:
        acc = (logits.argmax(1) == y_train).float().mean()
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}, Accuracy: {acc:.4f}")

# Plot training curve
plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.title('Neural ODE Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.show()

## 2. Implicit Neural Representations (SIREN)

SIREN (Sinusoidal Representation Networks) uses periodic activation functions to learn continuous representations of signals.

In [ ]:
class SineLayer(nn.Module):
    """Sine activation layer for SIREN"""
    def __init__(self, in_features, out_features, bias=True, is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        self.init_weights()
    
    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 1 / self.in_features)
            else:
                self.linear.weight.uniform_(
                    -np.sqrt(6 / self.in_features) / self.omega_0,
                    np.sqrt(6 / self.in_features) / self.omega_0
                )
    
    def forward(self, x):
        return torch.sin(self.omega_0 * self.linear(x))

class SIREN(nn.Module):
    """SIREN: Sinusoidal Representation Networks"""
    def __init__(self, in_features, hidden_features, hidden_layers, out_features, 
                 outermost_linear=True, first_omega_0=30, hidden_omega_0=1.):
        super().__init__()
        
        self.net = []
        self.net.append(SineLayer(in_features, hidden_features, 
                                 is_first=True, omega_0=first_omega_0))
        
        for i in range(hidden_layers):
            self.net.append(SineLayer(hidden_features, hidden_features, 
                                     is_first=False, omega_0=hidden_omega_0))
        
        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)
            with torch.no_grad():
                final_linear.weight.uniform_(
                    -np.sqrt(6 / hidden_features) / hidden_omega_0,
                    np.sqrt(6 / hidden_features) / hidden_omega_0
                )
            self.net.append(final_linear)
        else:
            self.net.append(SineLayer(hidden_features, out_features, 
                                     is_first=False, omega_0=hidden_omega_0))
        
        self.net = nn.Sequential(*self.net)
    
    def forward(self, coords):
        return self.net(coords)

In [ ]:
# Create image fitting task
def create_image_coords(height, width):
    """Create coordinate grid for image"""
    coords = torch.stack(torch.meshgrid(
        torch.linspace(-1, 1, height),
        torch.linspace(-1, 1, width),
        indexing='ij'
    ), dim=-1)
    return coords.reshape(-1, 2)

# Generate synthetic image
height, width = 64, 64
coords = create_image_coords(height, width)

# Create target image (checkerboard pattern)
target_image = ((coords[:, 0] * 5).sin() > 0) ^ ((coords[:, 1] * 5).sin() > 0)
target_image = target_image.float().reshape(height, width)

# Display target image
plt.figure(figsize=(6, 6))
plt.imshow(target_image, cmap='gray')
plt.title('Target Image')
plt.axis('off')
plt.show()

In [ ]:
# Train SIREN to fit image
print("Training SIREN to fit image...")
siren = SIREN(in_features=2, hidden_features=256, hidden_layers=3, 
              out_features=1).to(device)
optimizer = optim.Adam(siren.parameters(), lr=1e-4)

coords_train = coords.to(device)
target_train = target_image.reshape(-1, 1).to(device)

# Training loop with visualization
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
training_steps = [0, 200, 500, 1000]
step_idx = 0

for step in range(1001):
    optimizer.zero_grad()
    pred = siren(coords_train)
    loss = F.mse_loss(pred, target_train)
    loss.backward()
    optimizer.step()
    
    if step in training_steps:
        with torch.no_grad():
            pred_image = siren(coords_train).cpu().reshape(height, width)
            axes[step_idx].imshow(pred_image, cmap='gray')
            axes[step_idx].set_title(f'Step {step}')
            axes[step_idx].axis('off')
            step_idx += 1
        print(f"Step {step}, Loss: {loss.item():.6f}")

plt.suptitle('SIREN Image Fitting Progress')
plt.tight_layout()
plt.show()

## 3. Self-Supervised Learning (SimCLR)

SimCLR is a contrastive learning framework that learns representations without labels by maximizing agreement between differently augmented views of the same data.

In [ ]:
class SimCLR(nn.Module):
    """Simplified SimCLR for self-supervised learning"""
    def __init__(self, encoder_dim=128, projection_dim=64):
        super().__init__()
        # Simple encoder
        self.encoder = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, encoder_dim)
        )
        
        # Projection head
        self.projection = nn.Sequential(
            nn.Linear(encoder_dim, projection_dim),
            nn.ReLU(),
            nn.Linear(projection_dim, projection_dim)
        )
        
    def forward(self, x):
        h = self.encoder(x)
        z = self.projection(h)
        return h, z

def nt_xent_loss(z1, z2, temperature=0.5):
    """NT-Xent loss for contrastive learning"""
    batch_size = z1.shape[0]
    
    # Normalize embeddings
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)
    
    # Compute similarity matrix
    representations = torch.cat([z1, z2], dim=0)
    similarity_matrix = F.cosine_similarity(
        representations.unsqueeze(1), 
        representations.unsqueeze(0), 
        dim=2
    )
    
    # Create positive mask
    labels = torch.cat([torch.arange(batch_size), torch.arange(batch_size)], dim=0)
    labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
    labels = labels.to(device)
    
    # Mask out self-similarity
    mask = torch.eye(labels.shape[0], dtype=torch.bool).to(device)
    labels = labels[~mask].view(labels.shape[0], -1)
    similarity_matrix = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1)
    
    # Select positives
    positives = similarity_matrix[labels.bool()].view(labels.shape[0], -1)
    
    # Select negatives
    negatives = similarity_matrix[~labels.bool()].view(similarity_matrix.shape[0], -1)
    
    # Compute loss
    logits = torch.cat([positives, negatives], dim=1)
    labels = torch.zeros(logits.shape[0], dtype=torch.long).to(device)
    
    logits = logits / temperature
    return F.cross_entropy(logits, labels)

In [ ]:
# Data augmentation for self-supervised learning
def augment_data(x, noise_factor=0.1):
    """Simple augmentation by adding noise"""
    aug1 = x + torch.randn_like(x) * noise_factor
    aug2 = x + torch.randn_like(x) * noise_factor
    return aug1, aug2

# Train SimCLR
print("Training SimCLR...")
simclr_model = SimCLR().to(device)
optimizer = optim.Adam(simclr_model.parameters(), lr=0.001)

# Training loop
batch_size = 128
losses = []

for epoch in range(50):
    # Random data (in practice, use real images)
    x = torch.randn(batch_size, 784).to(device)
    
    # Create augmented views
    x1, x2 = augment_data(x)
    
    # Forward pass
    _, z1 = simclr_model(x1)
    _, z2 = simclr_model(x2)
    
    # Compute loss
    loss = nt_xent_loss(z1, z2)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Plot training curve
plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.title('SimCLR Training')
plt.xlabel('Epoch')
plt.ylabel('NT-Xent Loss')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Diffusion Models (Simplified)

Diffusion models generate data by gradually denoising random noise through a learned reverse diffusion process.

In [ ]:
class SimpleDiffusion(nn.Module):
    """Simplified diffusion model for 1D data"""
    def __init__(self, dim=2, time_dim=16):
        super().__init__()
        self.time_dim = time_dim
        
        # Time embedding
        self.time_mlp = nn.Sequential(
            nn.Linear(1, time_dim),
            nn.SiLU(),
            nn.Linear(time_dim, time_dim)
        )
        
        # Denoising network
        self.net = nn.Sequential(
            nn.Linear(dim + time_dim, 128),
            nn.SiLU(),
            nn.Linear(128, 128),
            nn.SiLU(),
            nn.Linear(128, dim)
        )
        
    def forward(self, x, t):
        # Embed time
        t_emb = self.time_mlp(t.unsqueeze(-1))
        # Concatenate with input
        h = torch.cat([x, t_emb], dim=-1)
        # Predict noise
        return self.net(h)

# Diffusion process utilities
def q_sample(x_0, t, noise=None, beta_schedule='linear', n_timesteps=100):
    """Forward diffusion process"""
    if noise is None:
        noise = torch.randn_like(x_0)
    
    # Simple linear schedule
    betas = torch.linspace(0.0001, 0.02, n_timesteps)
    alphas = 1.0 - betas
    alphas_cumprod = torch.cumprod(alphas, dim=0)
    
    # Get alpha values for timestep t
    sqrt_alphas_cumprod = alphas_cumprod.sqrt()
    sqrt_one_minus_alphas_cumprod = (1.0 - alphas_cumprod).sqrt()
    
    # Extract values for batch
    sqrt_alphas_cumprod_t = sqrt_alphas_cumprod[t].reshape(-1, 1)
    sqrt_one_minus_alphas_cumprod_t = sqrt_one_minus_alphas_cumprod[t].reshape(-1, 1)
    
    # Add noise
    return sqrt_alphas_cumprod_t * x_0 + sqrt_one_minus_alphas_cumprod_t * noise, noise

In [ ]:
# Generate 2D data (two moons)
from sklearn.datasets import make_moons
X_moons, _ = make_moons(n_samples=1000, noise=0.1)
X_moons = torch.FloatTensor(X_moons)

# Visualize data and diffusion process
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Original data
axes[0].scatter(X_moons[:, 0], X_moons[:, 1], alpha=0.5)
axes[0].set_title('Original Data')
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')

# Show diffusion process at different timesteps
timesteps = [10, 50, 99]
for i, t in enumerate(timesteps):
    x_noisy, _ = q_sample(X_moons[:100], torch.tensor([t]*100), n_timesteps=100)
    axes[i+1].scatter(x_noisy[:, 0], x_noisy[:, 1], alpha=0.5)
    axes[i+1].set_title(f'Timestep {t}')
    axes[i+1].set_xlabel('X')
    axes[i+1].set_ylabel('Y')

plt.suptitle('Forward Diffusion Process')
plt.tight_layout()
plt.show()

In [ ]:
# Train diffusion model
print("Training simplified diffusion model...")
diffusion_model = SimpleDiffusion(dim=2).to(device)
optimizer = optim.Adam(diffusion_model.parameters(), lr=0.001)

n_timesteps = 100
batch_size = 64
losses = []

for epoch in range(200):
    # Sample batch
    idx = torch.randint(0, len(X_moons), (batch_size,))
    x_0 = X_moons[idx].to(device)
    
    # Sample timesteps
    t = torch.randint(0, n_timesteps, (batch_size,)).to(device)
    
    # Add noise
    x_t, noise = q_sample(x_0, t, n_timesteps=n_timesteps)
    
    # Predict noise
    predicted_noise = diffusion_model(x_t, t.float() / n_timesteps)
    
    # MSE loss
    loss = F.mse_loss(predicted_noise, noise)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if epoch % 40 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Plot training curve
plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.title('Diffusion Model Training')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.grid(True, alpha=0.3)
plt.show()

## 5. Advanced Transformer Techniques

Exploring recent innovations in transformer architectures including RoPE, Flash Attention, and Mixture of Experts.

In [ ]:
class RotaryPositionalEmbedding(nn.Module):
    """Rotary Positional Embedding (RoPE)"""
    def __init__(self, dim, max_seq_len=1024):
        super().__init__()
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
        t = torch.arange(max_seq_len).float()
        freqs = torch.outer(t, inv_freq)
        self.register_buffer('cos', freqs.cos())
        self.register_buffer('sin', freqs.sin())
    
    def forward(self, x, seq_len=None):
        if seq_len is None:
            seq_len = x.shape[1]
        
        cos = self.cos[:seq_len, :]
        sin = self.sin[:seq_len, :]
        
        # Apply rotation
        x1, x2 = x.chunk(2, dim=-1)
        x_rot = torch.cat([-x2, x1], dim=-1)
        
        x_pos = x * cos + x_rot * sin
        return x_pos

class MoELayer(nn.Module):
    """Mixture of Experts layer"""
    def __init__(self, dim, num_experts=4, expert_capacity=2):
        super().__init__()
        self.num_experts = num_experts
        self.expert_capacity = expert_capacity
        
        # Gate network
        self.gate = nn.Linear(dim, num_experts)
        
        # Experts
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(dim, 4 * dim),
                nn.ReLU(),
                nn.Linear(4 * dim, dim)
            ) for _ in range(num_experts)
        ])
        
    def forward(self, x):
        batch_size, seq_len, dim = x.shape
        
        # Compute gates
        gates = F.softmax(self.gate(x), dim=-1)
        
        # Top-k routing
        topk_gates, topk_indices = gates.topk(2, dim=-1)
        topk_gates = topk_gates / topk_gates.sum(dim=-1, keepdim=True)
        
        # Process through experts (simplified)
        output = torch.zeros_like(x)
        for i in range(self.num_experts):
            expert_mask = (topk_indices == i).any(dim=-1)
            if expert_mask.any():
                expert_input = x[expert_mask]
                expert_output = self.experts[i](expert_input)
                
                # Weighted combination
                expert_gates = topk_gates[expert_mask]
                expert_gates = expert_gates[topk_indices[expert_mask] == i].unsqueeze(-1)
                
                output[expert_mask] += expert_gates * expert_output
        
        return output

In [ ]:
# Demonstrate advanced transformer components
print("Testing advanced transformer components...")

# RoPE
rope = RotaryPositionalEmbedding(dim=64)
x = torch.randn(2, 10, 64)  # [batch, seq_len, dim]
x_pos = rope(x)
print(f"RoPE output shape: {x_pos.shape}")

# MoE
moe = MoELayer(dim=64, num_experts=4)
moe_out = moe(x)
print(f"MoE output shape: {moe_out.shape}")

# Visualize expert routing
with torch.no_grad():
    gates = F.softmax(moe.gate(x[0]), dim=-1)  # First sample
    
plt.figure(figsize=(10, 6))
plt.imshow(gates.numpy().T, aspect='auto', cmap='hot')
plt.colorbar(label='Gate Probability')
plt.xlabel('Sequence Position')
plt.ylabel('Expert ID')
plt.title('Expert Routing Visualization')
plt.show()

## 6. Emerging Techniques: HyperNetworks and Adaptive Architectures

Exploring networks that generate other networks and architectures that adapt based on input.

In [ ]:
class HyperNetwork(nn.Module):
    """Network that generates weights for another network"""
    def __init__(self, z_dim=10, main_input_dim=2, main_hidden_dim=32, main_output_dim=1):
        super().__init__()
        self.z_dim = z_dim
        self.main_dims = [main_input_dim, main_hidden_dim, main_output_dim]
        
        # Calculate total parameters needed
        total_params = 0
        for i in range(len(self.main_dims) - 1):
            total_params += self.main_dims[i] * self.main_dims[i+1]
            total_params += self.main_dims[i+1]  # biases
        
        # Hypernetwork that generates main network weights
        self.hypernet = nn.Sequential(
            nn.Linear(z_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, total_params)
        )
        
    def forward(self, z, x):
        # Generate weights
        params = self.hypernet(z)
        
        # Extract weights and biases
        idx = 0
        for i in range(len(self.main_dims) - 1):
            w_size = self.main_dims[i] * self.main_dims[i+1]
            b_size = self.main_dims[i+1]
            
            w = params[idx:idx+w_size].view(self.main_dims[i+1], self.main_dims[i])
            b = params[idx+w_size:idx+w_size+b_size]
            idx += w_size + b_size
            
            # Apply layer
            x = F.linear(x, w, b)
            if i < len(self.main_dims) - 2:
                x = F.relu(x)
        
        return x

# Test hypernetwork
hypernet = HyperNetwork()
z = torch.randn(1, 10)  # Conditioning vector
x = torch.randn(5, 2)   # Input data
output = hypernet(z, x)
print(f"HyperNetwork output shape: {output.shape}")

# Visualize how different z values produce different functions
x_test = torch.linspace(-2, 2, 100).unsqueeze(1)
x_test = torch.cat([x_test, torch.zeros_like(x_test)], dim=1)  # 2D input

plt.figure(figsize=(10, 6))
for i in range(5):
    z = torch.randn(1, 10)
    y = hypernet(z, x_test)
    plt.plot(x_test[:, 0], y.detach().numpy(), label=f'Function {i+1}')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Different Functions Generated by HyperNetwork')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Summary and Research Directions

This tutorial covered several cutting-edge research topics in deep learning. Let's summarize the key concepts and future directions.

In [ ]:
# Create a comprehensive visualization of all techniques
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Neural ODE trajectories
ax = axes[0, 0]
with torch.no_grad():
    # Sample points
    x_test = torch.randn(100, 2) * 2
    x_encoded = ode_model.encoder(x_test)
    
    # Get intermediate states
    t = torch.linspace(0, 1, 20)
    trajectory = odeint(ode_model.ode.func, x_encoded, t, method='dopri5')
    
    # Plot some trajectories
    for i in range(5):
        traj = trajectory[:, i, :2].numpy()  # First 2 dimensions
        ax.plot(traj[:, 0], traj[:, 1], alpha=0.7)
    
ax.set_title('Neural ODE Trajectories')
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')

# SIREN frequency analysis
ax = axes[0, 1]
with torch.no_grad():
    # Get SIREN activations
    activations = []
    def hook(module, input, output):
        activations.append(output.detach().cpu())
    
    handle = siren.net[0].register_forward_hook(hook)
    _ = siren(coords[:100])
    handle.remove()
    
    # Plot activation histogram
    ax.hist(activations[0].numpy().flatten(), bins=50, alpha=0.7)
    ax.set_title('SIREN Activation Distribution')
    ax.set_xlabel('Activation Value')
    ax.set_ylabel('Frequency')

# Self-supervised representations
ax = axes[0, 2]
with torch.no_grad():
    # Generate random data
    x_test = torch.randn(200, 784).to(device)
    representations, _ = simclr_model(x_test)
    representations = representations.cpu().numpy()
    
    # Simple 2D projection for visualization
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2)
    rep_2d = pca.fit_transform(representations)
    
    ax.scatter(rep_2d[:, 0], rep_2d[:, 1], alpha=0.6)
    ax.set_title('Self-Supervised Representations')
    ax.set_xlabel('PC 1')
    ax.set_ylabel('PC 2')

# Diffusion process
ax = axes[1, 0]
# Show original data
ax.scatter(X_moons[:, 0], X_moons[:, 1], alpha=0.3, label='Original', s=20)

# Show noisy data at different timesteps
for t in [25, 50, 75]:
    x_noisy, _ = q_sample(X_moons[:100], torch.tensor([t]*100), n_timesteps=100)
    ax.scatter(x_noisy[:, 0], x_noisy[:, 1], alpha=0.3, 
               label=f't={t}', s=10)

ax.set_title('Diffusion Process')
ax.legend()
ax.set_xlabel('X')
ax.set_ylabel('Y')

# Research timeline
ax = axes[1, 1]
years = [2018, 2019, 2020, 2021, 2022, 2023]
topics = ['BERT', 'GPT-2', 'ViT', 'DALL-E', 'ChatGPT', 'GPT-4']
impact = [85, 88, 82, 90, 95, 98]

ax.plot(years, impact, 'o-', markersize=10, linewidth=2)
for i, txt in enumerate(topics):
    ax.annotate(txt, (years[i], impact[i]), 
                xytext=(5, 5), textcoords='offset points')

ax.set_title('Recent AI Breakthroughs')
ax.set_xlabel('Year')
ax.set_ylabel('Impact Score')
ax.grid(True, alpha=0.3)

# Research areas comparison
ax = axes[1, 2]
areas = ['Neural\nODEs', 'Implicit\nNeural\nReps', 'Self-\nSupervised', 
         'Diffusion\nModels', 'Advanced\nTransformers']
complexity = [7, 6, 5, 8, 9]
impact = [6, 7, 9, 10, 10]

ax.scatter(complexity, impact, s=200, alpha=0.6)
for i, txt in enumerate(areas):
    ax.annotate(txt, (complexity[i], impact[i]), 
                ha='center', va='center', fontsize=8)

ax.set_xlabel('Implementation Complexity')
ax.set_ylabel('Research Impact')
ax.set_title('Research Areas Comparison')
ax.grid(True, alpha=0.3)

plt.suptitle('Advanced Research Topics Overview', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Print summary
print("Advanced Research Topics Summary")
print("=" * 50)

research_areas = {
    "Neural ODEs": {
        "Key Idea": "Continuous-depth neural networks",
        "Advantages": "Memory efficient, continuous dynamics",
        "Applications": "Time series, physics simulation"
    },
    "Implicit Neural Representations": {
        "Key Idea": "Coordinate-based networks",
        "Advantages": "Continuous, resolution-agnostic",
        "Applications": "3D reconstruction, image compression"
    },
    "Self-Supervised Learning": {
        "Key Idea": "Learning without labels",
        "Advantages": "Leverages unlabeled data",
        "Applications": "Representation learning, pretraining"
    },
    "Diffusion Models": {
        "Key Idea": "Generation via denoising",
        "Advantages": "High quality, stable training",
        "Applications": "Image generation, audio synthesis"
    },
    "Advanced Transformers": {
        "Key Idea": "Efficient attention mechanisms",
        "Advantages": "Scalability, performance",
        "Applications": "NLP, computer vision, multimodal"
    }
}

for area, details in research_areas.items():
    print(f"\n{area}:")
    for key, value in details.items():
        print(f"  {key}: {value}")

print("\nFuture Directions:")
print("- Neuromorphic computing")
print("- Quantum machine learning")
print("- Causal representation learning")
print("- Continual learning")
print("- Energy-efficient AI")
print("- Interpretable AI")

print("\nKey Takeaways:")
print("- Research is rapidly evolving")
print("- Cross-pollination between fields is common")
print("- Theory and practice go hand in hand")
print("- Open problems abound")
print("- The field needs diverse perspectives")